In [3]:
import pandas as pd

dsp_email_list = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\DSP and BC Email List.xlsx',sheet_name = 0,usecols = ('DSP','DSP NAME','email_id') )
dsp_email_list.rename(columns={'email_id':'dsp_emailid'}, inplace=True)


station_email_list = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\DSP and BC Email List.xlsx',usecols = ('Delivery Station','Concat'),sheet_name = 1)
station_email_list.rename(columns={'Concat':'station_emailid'}, inplace=True)

# dsp_email_list
# station_email_list

invalid_data = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\FDD Email Disposition.xlsx')

invalid_dsps = list(invalid_data['DSP CODE'].unique())

valid_data = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\DSPFleetAuditcount.xlsx')

rates_data = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\Rate Calculation.xlsx', usecols=('DS','Monthly Van Rent'))
rates_data['Week Rent']=(rates_data['Monthly Van Rent']/30)*7


registration_data = pd.read_excel(r'C:\Users\majidmm\Documents\FDD\Input files templates\Fleet_Aggregated_View_data.xlsx',usecols=('Week','DSP','Station','Authorized Fleet','Capped Registered Fleet'))
registration_data.rename(columns={'DSP':'DSP CODE'}, inplace=True)
registration_data.rename(columns={'Station':'STATION CODE'}, inplace=True)
registration_data.rename(columns={'Authorized Fleet':'Authorized Vehicle count'}, inplace=True)
registration_data.rename(columns={'Capped Registered Fleet':'Registered Vehicle Count'}, inplace=True)
registration_data['Diff']=(registration_data['Authorized Vehicle count']-registration_data['Registered Vehicle Count'])
registration_data

rate_calculation=registration_data.merge(rates_data,left_on='STATION CODE',right_on='DS',how='left')
rate_calculation['Impact']=rate_calculation['Diff']*rate_calculation['Week Rent']
rate_calculation['Impact']=rate_calculation['Impact'].round(0)
rate_calculation

# # Set-1 outputs
invalid_data = invalid_data.merge(dsp_email_list,left_on='DSP CODE',right_on='DSP',how='left')
invalid_data = invalid_data.merge(station_email_list,left_on='STATION CODE',right_on='Delivery Station',how='left')
invalid_data

invalid_data[invalid_data['dsp_emailid'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\no_dsp_emailid.xlsx",index=False, header=True)
invalid_data[invalid_data['station_emailid'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\no_station_emailid.xlsx",index=False, header=True)
#invalid_data[invalid_data['DSP NAME'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\no_dsp_name.xlsx",index=False, header=True)
#invalid_data[invalid_data['dsp_emailid'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\no_dsp_emailid.xlsx",index=False, header=True)

invalid_data_final = invalid_data[invalid_data['DSP'].notnull()]
invalid_data_final['email_id']=invalid_data_final[['dsp_emailid','station_emailid']].apply(lambda x: ','.join(x.dropna()), axis=1)
invalid_data_final['email_id']=invalid_data_final['email_id']+','+'sisathee@amazon.com'
invalid_data_final['RENTAL OUT DATE'] = pd.to_datetime(invalid_data_final['RENTAL OUT DATE'],errors='coerce').dt.date
invalid_data_final['RENTAL IN DATE'] = pd.to_datetime(invalid_data_final['RENTAL IN DATE'],errors='coerce').dt.date
invalid_data_final['SUBMISSION DATE'] = pd.to_datetime(invalid_data_final['SUBMISSION DATE'],errors='coerce').dt.date
invalid_dsps_final = list(invalid_data_final['DSP CODE'].unique())

invalid_data_final.to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\Invalid_receipts_data.xlsx",index=False, header=True, columns=['DSP CODE','DSP NAME','STATION CODE','LICENSE NUMBER','RENTAL OUT DATE','RENTAL IN DATE','SUBMISSION DATE','VIN','FILE TITLE','STATUS','REJECTION REASON','SUBMISSION WEEK','email_id'])

invalid_rate_calculation = rate_calculation[rate_calculation['DSP CODE'].isin(invalid_dsps_final)]
invalid_rate_calculation.to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\Registration_data.xlsx",index=False, header=True,columns=['Week','DSP CODE','STATION CODE','Authorized Vehicle count','Registered Vehicle Count','Impact'])


invalid_valid_disputes_data = valid_data[valid_data['DSP CODE'].isin(invalid_dsps_final)]
invalid_valid_disputes_data['Date'] = pd.to_datetime(invalid_valid_disputes_data['Date'],errors='coerce').dt.date
invalid_valid_disputes_data.to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-1\Valid_receipts_data.xlsx",index=False, header=True)


#Set-2 outputs

set2_registration_data=rate_calculation[~rate_calculation['DSP CODE'].isin(invalid_dsps)]

set2_registration_data=set2_registration_data.merge(dsp_email_list,left_on='DSP CODE',right_on='DSP',how='left')
set2_registration_data=set2_registration_data.merge(station_email_list,left_on='STATION CODE',right_on='Delivery Station',how='left')
set2_registration_data[set2_registration_data['dsp_emailid'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-2\no_dsp_emailid.xlsx",index=False, header=True)
set2_registration_data[set2_registration_data['station_emailid'].isnull()].to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-2\no_station_emailid.xlsx",index=False, header=True)
set2_registration_data = set2_registration_data[set2_registration_data['DSP'].notnull()]
set2_registration_data['email_id']=set2_registration_data[['dsp_emailid','station_emailid']].apply(lambda x: ','.join(x.dropna()), axis=1)
set2_registration_data['email_id']=set2_registration_data['email_id']+','+'sisathee@amazon.com'
set2_registration_data.to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-2\Registration_data.xlsx",index=False, header=True,columns=['Week','DSP CODE','STATION CODE','Authorized Vehicle count','Registered Vehicle Count','Impact','DSP NAME','email_id'])

set2_dsps = list(set2_registration_data['DSP CODE'].unique())
set2_dsps

set2_valid_disputes_data = valid_data[valid_data['DSP CODE'].isin(set2_dsps)]
set2_valid_disputes_data['Date'] = pd.to_datetime(set2_valid_disputes_data['Date'],errors='coerce').dt.date

set2_valid_disputes_data.to_excel(r"C:\Users\majidmm\Documents\FDD\Input files templates\output\Set-2\Valid_receipts_data.xlsx",index=False, header=True)
set2_valid_disputes_data


<ipython-input-3-b613d600e77d>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_data_final['email_id']=invalid_data_final[['dsp_emailid','station_emailid']].apply(lambda x: ','.join(x.dropna()), axis=1)
<ipython-input-3-b613d600e77d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invalid_data_final['email_id']=invalid_data_final['email_id']+','+'sisathee@amazon.com'
<ipython-input-3-b613d600e77d>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,DSP CODE,Week,STATION CODE,Date,Valid Receipt Count
244,1PRI,36,DMI7,2020-09-01,12
245,1PRI,36,DMI7,2020-09-02,12
246,1PRI,36,DMI7,2020-09-03,12
247,1PRI,36,DMI7,2020-09-04,12
248,1PRI,36,DMI7,2020-09-05,12
...,...,...,...,...,...
94850,PINP,44,DGE9,2020-10-27,0
94851,PINP,44,DGE9,2020-10-28,0
94852,PINP,44,DGE9,2020-10-29,0
94853,PINP,44,DGE9,2020-10-30,0
